# Format Data - Gen Baseline

In [1]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import glob as gb

In [ ]:
main_condition = 'Generation'
model_name = 'gpt3-text-davinci-002' #'gpt3-text-davinci-002', 'bert-large-cased'
tk_model_name = 'gpt2' #'bigscience/bloom' #model_name#'bigscience/bloom-7b1'

prime_pause_str = '.'

sub_conditions = ['baseline', 'baseline50', 'all', 'all_32','100_64_20_5','10_64_20_5','100_64_20_1']

exp_num = 1

path = '/Users/user/ECAI/Results/'

path += f'{main_condition}/{model_name}/{sub_conditions[exp_num-1]}/'

path

In [12]:
tokenizer = AutoTokenizer.from_pretrained(tk_model_name)

AAB_prime = pd.DataFrame([])
ABA_prime = pd.DataFrame([])
ABB_prime = pd.DataFrame([])
ABCa_prime = pd.DataFrame([])
ABCb_prime = pd.DataFrame([])
df = pd.DataFrame([])

all_files = [[],[],[],[],[]]
all_files[0] = gb.glob(path + '*generate_AAB.csv') # 0 = AAB primes
all_files[1] = gb.glob(path + '*generate_ABA.csv') # 1 = ABA primes
all_files[2] = gb.glob(path + '*generate_ABB.csv') # 2 = ABB primes
all_files[3] = gb.glob(path + '*generate_ABCa.csv') # 3 = ABCa primes
all_files[4] = gb.glob(path + '*generate_ABCb.csv') # 4 = ABCb primes

for sub_files in all_files:
    for filename in sub_files:
        
        df = pd.read_csv(filename, index_col=0, header=0, sep=';',)
        
        if filename.find('generate_AAB.csv') != -1: AAB_prime = AAB_prime.append(df)
        elif filename.find('generate_ABA.csv') != -1: ABA_prime = ABA_prime.append(df)
        elif filename.find('generate_ABB.csv') != -1: ABB_prime = ABB_prime.append(df)
        elif filename.find('generate_ABCa.csv') != -1: ABCa_prime = ABCa_prime.append(df)
        elif filename.find('generate_ABCb.csv') != -1: ABCb_prime = ABCb_prime.append(df)
        else: print('Error')

#print(all_files)

""" Reformat to np.array for Top 1 Outputs """
for i in range(1,3,1):
    AAB_prime.iloc[:,i] = AAB_prime.iloc[:,i].apply(lambda x: np.fromstring(x[1:-1], dtype=np.int64, sep=', '))
    ABA_prime.iloc[:,i] = ABA_prime.iloc[:,i].apply(lambda x: np.fromstring(x[1:-1], dtype=np.int64, sep=', '))
    ABB_prime.iloc[:,i] = ABB_prime.iloc[:,i].apply(lambda x: np.fromstring(x[1:-1], dtype=np.int64, sep=', '))

    ABCa_prime.iloc[:,i] = ABCa_prime.iloc[:,i].apply(lambda x: np.fromstring(x[1:-1], dtype=np.int64, sep=', '))
    ABCb_prime.iloc[:,i] = ABCb_prime.iloc[:,i].apply(lambda x: np.fromstring(x[1:-1], dtype=np.int64, sep=', '))
    
    if model_name.lower().find('bert')>-1: # Remove special tokens
        AAB_prime.iloc[:,i] = AAB_prime.iloc[:,i].apply(lambda x: x[1:-1])
        ABA_prime.iloc[:,i] = ABA_prime.iloc[:,i].apply(lambda x: x[1:-1])
        ABB_prime.iloc[:,i] = ABB_prime.iloc[:,i].apply(lambda x: x[1:-1])
        ABCa_prime.iloc[:,i] = ABCa_prime.iloc[:,i].apply(lambda x: x[1:-1])
        ABCb_prime.iloc[:,i] = ABCb_prime.iloc[:,i].apply(lambda x: x[1:-1])

def ASR_checker(ASR_output):
    ASR_output = ASR_output.iloc[:,:3].iterrows()
    count_AAB = 0
    count_ABA = 0
    count_ABB = 0
    count_ABC = 0
    count_other = 0
    count_unAAB = 0
    count_unABA = 0
    count_unABB = 0
    count_unABC = 0
    count_UNother = 0
        
    for row in ASR_output:
        setting = row[1][0]
        if setting == 'no-probe': inpt = row[1][1]
        else: inpt = row[1][1][:-1]
        check = row[1][2:3]
        #print(len(check), check)
        if check[0].size < 3:
            count_other +=1
            #print('other<: ', check)
        else: 
            check = check[0]
            #print(check, check[0])

            if check[0]==check[1] and check[1]!=check[2]: count_AAB += 1
            elif check[0]==check[2] and check[2]!=check[1]: count_ABA += 1
            elif check[1]==check[2] and check[2]!=check[0]: count_ABB += 1
            elif check[0]!=check[1] and check[1]!=check[2] and check[2]!=check[0]: count_ABC += 1
            else: 
                count_other += 1
                print('__________')
                print('__________')
                print('other: ', check)
                print('__________')
                print('__________')
            
            no_copy = True
            #print(inpt, len(inpt))
            for sub in np.reshape(inpt,(len(inpt)//4,-1)):
                if tokenizer.convert_ids_to_tokens(int(sub[0])).lower() == tokenizer.convert_ids_to_tokens(int(check[0])).lower():
                    if tokenizer.convert_ids_to_tokens(int(sub[1])).lower() == tokenizer.convert_ids_to_tokens(int(check[1])).lower():
                        if tokenizer.convert_ids_to_tokens(int(sub[2])).lower() == tokenizer.convert_ids_to_tokens(int(check[2])).lower():
                            no_copy = False
                            print(check, sub, no_copy)
                            print(tokenizer.convert_ids_to_tokens(int(sub[0])).lower(), tokenizer.convert_ids_to_tokens(int(check[0])).lower())
                            print(tokenizer.convert_ids_to_tokens(int(sub[1])).lower(), tokenizer.convert_ids_to_tokens(int(check[1])).lower())
                            print(tokenizer.convert_ids_to_tokens(int(sub[2])).lower(), tokenizer.convert_ids_to_tokens(int(check[2])).lower())
                            print('__________')
            
            if no_copy: #set(inpt).intersection(set(check)) == set(): 
                if check[0]==check[1] and check[1]!=check[2]: count_unAAB += 1
                elif check[0]==check[2] and check[2]!=check[1]: count_unABA += 1
                elif check[1]==check[2] and check[2]!=check[0]: count_unABB += 1
                elif check[0]!=check[1] and check[1]!=check[2] and check[2]!=check[0]: count_unABC += 1
                else: 
                    count_UNother += 1
                    print('__________')
                    print('__________')
                    print('otherUN: ', check)
                    print('__________')
                    print('__________')
    
    return np.array([count_AAB,count_ABA,count_ABB, count_ABC, count_other,
                     count_unAAB,count_unABA,count_unABB,count_unABC,count_UNother])

cols = ['probeAAB', 'probeABA', 'probeABB', 'probeABC', 'probeOTHER', 'UNprobeAAB', 'UNprobeABA', 'UNprobeABB', 'UNprobeABC', 'UNprobeOTHER']

all_data = np.zeros((5, len(cols)))
all_data[0] = ASR_checker(AAB_prime[AAB_prime['Setting'].str.contains('no')])
all_data[1] = ASR_checker(ABA_prime[ABA_prime['Setting'].str.contains('no')])
all_data[2] = ASR_checker(ABB_prime[ABB_prime['Setting'].str.contains('no')])
all_data[3] = ASR_checker(ABCa_prime[ABCa_prime['Setting'].str.contains('no')])
all_data[4] = ASR_checker(ABCb_prime[ABCb_prime['Setting'].str.contains('no')])

csv_data = pd.DataFrame(all_data, columns=cols, index=['prime AAB', 'prime ABA', 'prime ABB', 'prime ABCa', 'prime ABCb'])

# seen & unseen
csv_data.T.iloc[:5,:].rename(index={'probeAAB':'AAB Pattern Outputs', 'probeABA':'ABA Pattern Outputs', 
                                    'probeABB':'ABB Pattern Outputs', 'probeABC':'ABC Pattern Outputs',
                                    'probeOTHER':'Other Outputs'}
                            ).to_csv(path + model_name + '_allSU.csv')

# unseen
csv_data.T.iloc[5:,:].rename(index={'UNprobeAAB':'AAB Pattern Outputs', 'UNprobeABA':'ABA Pattern Outputs', 
                                    'UNprobeABB':'ABB Pattern Outputs', 'UNprobeABC':'ABC Pattern Outputs',
                                    'UNprobeOTHER':'Other Outputs'}
                            ).to_csv(path + model_name + '_allU.csv')

all_data = np.zeros((5, len(cols)))
all_data[0] = ASR_checker(AAB_prime[AAB_prime['Setting'].str.contains('A')])
all_data[1] = ASR_checker(ABA_prime[ABA_prime['Setting'].str.contains('A')])
all_data[2] = ASR_checker(ABB_prime[ABB_prime['Setting'].str.contains('A')])
all_data[3] = ASR_checker(ABCa_prime[ABCa_prime['Setting'].str.contains('A')])
all_data[4] = ASR_checker(ABCb_prime[ABCb_prime['Setting'].str.contains('A')])

csv_data = pd.DataFrame(all_data, columns=cols, index=['prime AAB', 'prime ABA', 'prime ABB', 'prime ABCa', 'prime ABCb'])

# seen & unseen
csv_data.T.iloc[:5,:].rename(index={'probeAAB':'AAB Pattern Outputs', 'probeABA':'ABA Pattern Outputs', 
                                    'probeABB':'ABB Pattern Outputs', 'probeABC':'ABC Pattern Outputs',
                                    'probeOTHER':'Other Outputs'}
                            ).to_csv(path + model_name + '_allSUf.csv')

# unseen
csv_data.T.iloc[5:,:].rename(index={'UNprobeAAB':'AAB Pattern Outputs', 'UNprobeABA':'ABA Pattern Outputs', 
                                    'UNprobeABB':'ABB Pattern Outputs', 'UNprobeABC':'ABC Pattern Outputs',
                                    'UNprobeOTHER':'Other Outputs'}
                            ).to_csv(path + model_name + '_allUf.csv')

#csv_data.to_csv(path + 'success_rate.csv')

[4526 4526 8696] [ 797  797 8696   13] False
ġre ġre
ġre ġre
ġev ġev
__________
__________
__________
other:  [14496 14496 14496]
__________
__________
__________
__________
otherUN:  [14496 14496 14496]
__________
__________
__________
__________
other:  [5121 5121 5121]
__________
__________
__________
__________
otherUN:  [5121 5121 5121]
__________
__________
__________
__________
other:  [35480 35480 35480]
__________
__________
__________
__________
otherUN:  [35480 35480 35480]
__________
__________
[ 611 1846  611] [ 611 1846  611   13] False
ġif ġif
ġim ġim
ġif ġif
__________
[11948 13182 11948] [  502 13182   502    13] False
ġme ġme
ġec ġec
ġme ġme
__________
[35774 16543 35774] [35774 16543 35774    13] False
ġud ġud
ġom ġom
ġud ġud
__________
__________
__________
other:  [8252 8252 8252]
__________
__________
__________
__________
otherUN:  [8252 8252 8252]
__________
__________
[21628 21733 21628] [16975 14364 16975    13] False
ġiv ġiv
ġnu ġnu
ġiv ġiv
__________
[22631 

In [58]:
# seen & unseen
csv_data.T.iloc[:4,:].rename(index={'probeAAB':'AAB Pattern Outputs', 'probeABA':'ABA Pattern Outputs', 
                                    'probeABB':'ABB Pattern Outputs','probeOTHER':'Other Outputs'})

,prime AAB,prime ABA,prime ABB
AAB Pattern Outputs,0.72,0.04,0.0
ABA Pattern Outputs,0.26,0.56,0.0
ABB Pattern Outputs,0.00,0.00,0.5
Other Outputs,0.02,0.40,0.5


In [59]:
# unseen
csv_data.T.iloc[4:,:].rename(index={'UNprobeAAB':'AAB Pattern Outputs', 'UNprobeABA':'ABA Pattern Outputs', 
                                    'UNprobeABB':'ABB Pattern Outputs','UNprobeOTHER':'Other Outputs'})

,prime AAB,prime ABA,prime ABB
AAB Pattern Outputs,0.606061,0.03125,0.000000
ABA Pattern Outputs,0.393939,0.46875,0.000000
ABB Pattern Outputs,0.000000,0.00000,0.354839
Other Outputs,0.000000,0.50000,0.645161
